In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HistopathologyCNN(nn.Module):
    def __init__(self, num_classes=2, leaky_relu_slope=0.0):
        super(HistopathologyCNN, self).__init__()

        # Conv1: 3 input channels (RGB), 32 filters, 5x5 kernel, padding=2, stride=1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        nn.init.normal_(self.conv1.weight, mean=0.0, std=0.0001)  # Gaussian init std=0.0001

        # Conv2: 32 input channels, 32 filters
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)
        nn.init.normal_(self.conv2.weight, mean=0.0, std=0.01)    # std=0.01

        # Conv3: 32 input channels, 64 filters
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        nn.init.normal_(self.conv3.weight, mean=0.0, std=0.0001)  # std=0.0001

        # Pooling layers
        self.pool_max = nn.MaxPool2d(kernel_size=3, stride=2)
        self.pool_avg = nn.AvgPool2d(kernel_size=3, stride=2)

        # Activation function (LeakyReLU or ReLU)
        if leaky_relu_slope > 0:
            self.relu = nn.LeakyReLU(negative_slope=leaky_relu_slope)
        else:
            self.relu = nn.ReLU()

        # Fully connected layers
        # Need to calculate input size for first FC layer after conv+pool
        # Assuming input size 64x64 as per your patches:
        # After conv1 + maxpool (3x3 stride 2): output size = floor((64 + 2*2 - 5)/1 + 1) = 64 conv1 output, then pool = floor((64 - 3)/2 + 1) = 31
        # Similarly for conv2 + avgpool and conv3 + avgpool, calculate final feature map size:
        # Let's compute stepwise:

        # After conv1: size remains 64x64 (padding=2 keeps size)
        # After pool_max(3x3 stride 2): floor((64-3)/2)+1 = 31

        # After conv2: size remains 31x31
        # After pool_avg(3x3 stride 2): floor((31-3)/2)+1 = 15

        # After conv3: size remains 15x15
        # After pool_avg(3x3 stride 2): floor((15-3)/2)+1 = 7

        # So final feature map size: 64 channels × 7 × 7 = 3136

        self.fc1 = nn.Linear(64 * 7 * 7, 512)  # can adjust 512 as needed
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Conv1 + ReLU + MaxPool
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool_max(x)

        # Conv2 + ReLU + AvgPool
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool_avg(x)

        # Conv3 + ReLU + AvgPool
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool_avg(x)

        # Flatten for FC
        x = x.view(x.size(0), -1)

        # FC layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        # Output (no softmax here since we'll use CrossEntropyLoss which applies softmax)
        return x


In [11]:
from PIL import Image
import numpy as np
n=0
from torchvision import transforms
def patching(img_path):
    patched=[]
    img=Image.open(img_path).convert("RGB")
    img_array=np.array(img)
    for i in range(0,img_array.shape[0],64):
        for j in range(0,img_array.shape[1],64):
            patch=img_array[i:i+64,j:j+64]
            if(patch.shape[0]==64 and patch.shape[1]==64):
                # print(f"before cpatch{patch.shape}" )
                patch = np.transpose(patch, (2, 0, 1))
                patch=torch.tensor(patch)
                
                # print(f"after patching{patch.shape}" )
                
                patched.append(patch)
    return patched  
  

In [12]:
model=HistopathologyCNN()
model.load_state_dict(torch.load("breast_model_weights_20_epoch.pth",weights_only=True))
model.eval()

HistopathologyCNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool_max): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool_avg): AvgPool2d(kernel_size=3, stride=2, padding=0)
  (relu): ReLU()
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [20]:
from torchvision import transforms
transform = transforms.ToTensor()
from PIL import Image
img1 = transform(Image.open('mean_image_benign.png').convert('RGB'))
img2 = transform(Image.open('mean_image_malignant.png').convert('RGB'))
device="cuda"
# --- Step 2: Compute mean image ---
mean_image = (img1 + img2) / 2

def output(img_path,n,mean_image=mean_image):
    output_arr=[]
    for img in patching(img_path):
        # print(img.shape)
        final_img=(img-mean_image)
        final_img = final_img.unsqueeze(0)
        # print(f"final image shape {final_img.shape}")
        
        _, predicted = torch.max(model(final_img), 1)
        print(f"predict is {model(final_img)}")
        output_arr.append(predicted.cpu().numpy())
    total_sum = sum(output_arr)
    if((total_sum/n)>0.5):
        return 1   
    return 0

        


In [14]:
mean_image.shape

torch.Size([3, 64, 64])

In [15]:
import os
list=os.listdir("main_data/malignant")

In [16]:
len(list)

1372

In [17]:
import random
random_image = random.choice(list)

In [18]:
random_image
n=len(patching("main_data/malignant/"+random_image))
n

70

In [21]:
count=0
c=0
for k in range(100):
    random_image = random.choice(list)
    c=c+1
    if(output("main_data/malignant/"+random_image,n,mean_image)):
        count=count+1
    print(f"{c} image completed")

print(f"accuracy is {100-count}%")
    

predict is tensor([[ 5417.8506, -5362.0518]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5207.2583, -5151.6196]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 7492.0283, -7420.7139]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 6639.5903, -6570.3579]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 6062.6162, -6006.1460]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5956.4927, -5901.7661]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5429.3853, -5371.0098]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 4654.8271, -4609.6567]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5378.4272, -5328.2183]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5572.5483, -5512.1943]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5881.6045, -5826.9131]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5972.2876, -5904.1592]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 6979.1401, -6908.9717]], grad_fn=<AddmmBackward0>)
predict is tensor([[ 5963.5620, -5902.9692]], grad_fn=<AddmmBack

In [23]:
image=Image.open("1.png").convert("RGB")
image=np.array(image)
patch1 = np.transpose(image, (2, 0, 1))
img=torch.tensor(patch1)


final_img1=(img-mean_image)
final_img1= final_img1.unsqueeze(0)
model(final_img1)

tensor([[ 6211.7612, -6152.1997]], grad_fn=<AddmmBackward0>)